In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time

In [2]:
newdata=pd.read_csv("../input/records/records.csv")

# Preprocessing

In [3]:
all_text=newdata["transcription_text"]

In [4]:
all_text.iloc[:10000]

0       and one hundred dollars. I have never heard Ba...
1       The Freedmen's Bureau\r\nvs.\r\nB.M. Richardso...
2       The Freedmen's Bureau\r\nvs.\r\nB. M. Richards...
3       me one hundred dollars, two suits of clothes a...
4       questions again, and I gave him the same answe...
                              ...                        
9995    OFFICE SUPERINTENDENT OF SCHOOLS FOR FREEDMEN ...
9996    1. How many Day or Night Schools, within your ...
9997    O.S.F.S. Recd. Feb 9th 1867.\r\n\r\nMiss M. Sm...
9998    SCHOOL REPORT\r\nof Mary Trivinia Smith \r\nCo...
9999    REPORT OF SCHOOL FOR FREEDMEN.\r\n\r\nIn Milli...
Name: transcription_text, Length: 10000, dtype: object

In [5]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'

In [6]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s
start=time.time()
for i in tqdm(range(10000)):
    all_text[i] = process(all_text[i])
for i in tqdm(range(10000)):
    all_text[i] = " ".join(all_text[i])
end=time.time()
print(end-start)

100%|██████████| 10000/10000 [00:05<00:00, 1840.43it/s]

35.358251094818115


In [7]:
all_text

0        one hundred dollar . I never heard Bagby compl...
1        The Freedmens Bureau vs. B.M . Richardson Assa...
2        The Freedmens Bureau vs. B. M. Richardson , Co...
3        one hundred dollar , two suit of clothes would...
4        question , I gave answer , I whipped.— All I s...
                               ...                        
35163    648 Z\r\n\r\n[[7 columned table]]\r\n|Date.|Na...
35164    650\r\n\r\nDate. | Name of Male. | Name of Fem...
35165    Sarah Brame\r\nDied\r\nJune 25th 1847.\r\n\r\n...
35166    Frances S. Benners\r\n\r\nBorn\r\nAugust 30th ...
35167    State of North Carolina\r\nBladen County\r\n\r...
Name: transcription_text, Length: 35168, dtype: object

# Flair

In [8]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-py

# Hugging face  : Flair

**ner-english** 

In [9]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.38 MiB/s, done.


In [10]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-21 11:16:56,205 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-21 11:17:24,021 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [11]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-21 11:17:38,688 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-21 11:17:40,782 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [12]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-21 11:18:49,017 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-21 11:19:07,713 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [13]:
start=time.time()
listenglarge=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(10000):
        sentencess =Sentence(all_text[i])
        dflarge.loc[i,"text"]=all_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listenglarge.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG      
end=time.time()
print(end-start)

2022-06-21 11:21:02,466 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:21:07,006 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:21:26,382 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:24:28,808 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:24:59,494 Warning: An empty Sentence was created! Are there empty strings in your dataset?
460.4310882091522


In [14]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Richardson, W.M . Henry, J, Edward Hom...",[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,"[B.M, Richardson, Daniel Bagley Freedman, E. W...",[Raleigh],"[Freedmens Bureau, Bureau, Freedmens Court, Mi..."
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau]
3,"one hundred dollar , two suit of clothes would...",[Hawley],[],[]
4,"question , I gave answer , I whipped.— All I s...","[Bagley, Richardson]",[Martain County],[]
...,...,...,...,...
9995,OFFICE SUPERINTENDENT OF SCHOOLS FOR FREEDMEN ...,"[E.M, WHEELOCK]","[State of Texas, North]","[Freedmen of Texas, Schools Freedmen Refugees]"
9996,"1 . How many Day Night Schools , within knowle...",[],[],"[Bureau, Freedmen]"
9997,O.S.F.S . Recd . Feb 9th 1867 . Miss M. Smith ...,[Miss M. Smith Millican],[Texas],[O.S.F.S]
9998,SCHOOL REPORT of Mary Trivinia Smith County of...,[],[Brazos],[]


tagger1 ner english

In [15]:
start=time.time()
listeng=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dfner=pd.DataFrame({"text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(10000):
        sentencess =Sentence(all_text[i])
        dfner.loc[i,"text"]=all_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listeng.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dfner["PERSON-nerenglish"]=Per
dfner["LOC-nerenglish"]=LOC
dfner["ORG-nerenglish"]=ORG   
end=time.time()
print(end-start)

2022-06-21 11:34:32,486 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:34:48,780 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:36:00,972 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:48:03,741 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:50:04,195 Warning: An empty Sentence was created! Are there empty strings in your dataset?
1817.6361384391785


In [16]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Henry Signed, Richardsons, Edward Horn...",[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,[],[],[Freedmens Bureau]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau]
3,"one hundred dollar , two suit of clothes would...",[Genl Hawley],[],[]
4,"question , I gave answer , I whipped.— All I s...",[Bagley],[Martain County],[]


tagger 2 ontonotes

In [17]:
start=time.time()
listoto=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

aldf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(10000):
        sentencess =Sentence(all_text[i])
        aldf.loc[i,"text"]=all_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listoto.append(entity)
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
aldf["Date-ontonotes"]=Date
aldf["PERSON-ontonotes"]=Per
aldf["GPE-ontonotes"]=GPE
aldf["ORG-ontonotes"]=ORG   
end=time.time()
print(end-start)

2022-06-21 11:59:02,280 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:59:07,061 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 11:59:25,724 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 12:02:30,629 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-21 12:03:01,899 Warning: An empty Sentence was created! Are there empty strings in your dataset?
463.7297022342682


In [18]:
aldf.shape

(10000, 5)

In [19]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Richardson, W.M . Henry, H.H . Holton,...",[],[],"[March 19th 1866, March 19 1866]"
1,The Freedmens Bureau vs. B.M . Richardson Assa...,[B.M . Richardson],[Raleigh],"[The Freedmens Bureau, Freedmens Court, Milita...",[Mon 28 . 1866]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[The Freedmens Bureau],"[twenty one, twenty one year old]"
3,"one hundred dollar , two suit of clothes would...",[Genl Hawley],[],[],[]
4,"question , I gave answer , I whipped.— All I s...","[Bagley, Richardson]",[Martain County],[],[]


In [20]:
aldf.shape

(10000, 5)

In [21]:
len(Date)

10000

In [22]:
Date[15000:15020]

[]

In [23]:
import pickle

with open('Date.pkl', 'wb') as f:
    pickle.dump(Date, f)

# Post processing

**For Date**

In [24]:
G=[]
for i in aldf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
aldf["postDate-ontonotes"]=GF
            
            
        

# Run this to see the extracted dates before filtring them 

In [25]:
len(GF)

10000

In [26]:
GF[100:120]

[[],
 ['Tuesday Aug 1st 1865'],
 [],
 ['1865'],
 [],
 [],
 ['Jany.1.66', 'Feby.28,66'],
 ['Feb 28 1866'],
 [],
 [],
 ['Jany.1.66', 'Feby.28.66'],
 ['January 1 1866', 'Feb 28 1866'],
 [],
 [],
 [],
 [],
 [],
 ['Jan 1 1866'],
 [],
 ['Feb 28 1866']]

In [27]:
import pickle

with open('Datebeforefilter.pkl', 'wb') as f:
    pickle.dump(GF, f)

In [28]:
G=[]
for i in aldf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

aldf["postDate-ontonotes"]=G

In [29]:
len(G)

10000

In [30]:
G[8000:8020]

[['1-June-1867'],
 [],
 ['31-May-1867'],
 ['31-May-1867'],
 [],
 [],
 [],
 ['19-July-1867'],
 ['1-July-1867'],
 [],
 [],
 [],
 [],
 [],
 ['1-August-67'],
 ['1-August-67'],
 [],
 ['24-August-1867'],
 [],
 []]

In [31]:
import pickle

with open('Dateafterfilter.pkl', 'wb') as f:
    pickle.dump(G, f)

**great !**

****

****

In [32]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Richardson, W.M . Henry, J, Edward Hom...",[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,"[B.M, Richardson, Daniel Bagley Freedman, E. W...",[Raleigh],"[Freedmens Bureau, Bureau, Freedmens Court, Mi..."
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau]
3,"one hundred dollar , two suit of clothes would...",[Hawley],[],[]
4,"question , I gave answer , I whipped.— All I s...","[Bagley, Richardson]",[Martain County],[]


In [33]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Henry Signed, Richardsons, Edward Horn...",[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,[],[],[Freedmens Bureau]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau]
3,"one hundred dollar , two suit of clothes would...",[Genl Hawley],[],[]
4,"question , I gave answer , I whipped.— All I s...",[Bagley],[Martain County],[]


In [34]:
aldf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
aldf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
aldf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]
aldf["Person-NerEng"]=dfner["PERSON-nerenglish"]
aldf["LOC-NerEng"]=dfner["LOC-nerenglish"]
aldf["ORG-NerEng"]=dfner["ORG-nerenglish"]

In [35]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Richardson, W.M . Henry, H.H . Holton,...",[],[],"[March 19th 1866, March 19 1866]","[19-March-1866, 19-March-1866]","[Bagby, Richardson, W.M . Henry, J, Edward Hom...",[],[],"[Bagby, Henry Signed, Richardsons, Edward Horn...",[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,[B.M . Richardson],[Raleigh],"[The Freedmens Bureau, Freedmens Court, Milita...",[Mon 28 . 1866],[28-Mon-1866],"[B.M, Richardson, Daniel Bagley Freedman, E. W...",[Raleigh],"[Freedmens Bureau, Bureau, Freedmens Court, Mi...",[],[],[Freedmens Bureau]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[The Freedmens Bureau],"[twenty one, twenty one year old]",[],"[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau],"[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau]
3,"one hundred dollar , two suit of clothes would...",[Genl Hawley],[],[],[],[],[Hawley],[],[],[Genl Hawley],[],[]
4,"question , I gave answer , I whipped.— All I s...","[Bagley, Richardson]",[Martain County],[],[],[],"[Bagley, Richardson]",[Martain County],[],[Bagley],[Martain County],[]


In [36]:
allPerson=[]
for i in aldf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in aldf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in aldf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in aldf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in aldf["ORG-ontonotes"]:
            alORGl.append(i)

****

In [37]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)
ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i) 
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)

* as usual , ner English Large is better in extracting Person 
* ontonotes is thes best in extracting dates after good postprocessing and filtring almost all pattern 
* ner Engish Large  also better in Location and Oranization 

# dates

**run those cells to take a look at the result more clearly**

In [38]:
#G

In [39]:
import pickle

with open('finaldate.pkl', 'wb') as f:
    pickle.dump(G, f)

# Person

In [40]:
allPerson

[['Bagby',
  'Richardson',
  'W.M . Henry',
  'H.H . Holton',
  'Edward Homer',
  'Richardsons',
  'Edward Horner',
  'Mathew Baldwin',
  'Bagly',
  'Mathew'],
 ['B.M . Richardson'],
 ['B. M. Richardson', 'Daniel Bagley', 'Richardson'],
 ['Genl Hawley'],
 ['Bagley', 'Richardson'],
 ['Dan', 'Parker', 'Richardsons', 'B.M . Richardson'],
 ['Richardson', 'Clement Richardson'],
 ['R.'],
 ['Richardson'],
 ['Bagley', 'Richardson', 'R.', 'B.'],
 ['Parker', 'John A Bailey', 'Richardson', 'Bagley'],
 ['R.', 'Parker', 'Ben Simpson', 'Edward Horner', 'Dan Bagley'],
 ['Richardson', 'R.', 'W . T. Maultsby'],
 ['K . Haines', 'J . P. Johnson', 'Ben Simpson', 'Dan', 'R.'],
 ['Dan', 'R.', 'Barber', 'D.D . Barber'],
 ['R.', 'Dan', 'Mathew Baldwin', 'Dan Bagley'],
 ['Bagley', 'Dan', 'John Kelley'],
 ['R.', 'Rs', 'Dan Bagley', 'R', 'Dan'],
 ['Clement Richardson', 'B . M. Richardson', 'R.s', 'Dan'],
 ['Dan', 'Horner', 'R.', 'lobelia'],
 ['Dan', 'S.S . Ashley', 'R.', 'D. D. Barber', 'Ben Simpson'],
 ['Maulsb

In [41]:
import pickle

with open('Persononto.pkl', 'wb') as f:
    pickle.dump(allPerson, f)

In [42]:
 # Personlarge

In [43]:
import pickle

with open('Personlarge.pkl', 'wb') as f:
    pickle.dump(Personlarge, f)

# LOC and ORG 

In [44]:
#LOClarge
with open('LOClarge.pkl', 'wb') as f:
    pickle.dump(LOClarge, f)

In [45]:
#ORGlarge
with open('ORGlarge.pkl', 'wb') as f:
    pickle.dump(ORGlarge, f)

# the final dataset :

In [46]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,one hundred dollar . I never heard Bagby compl...,"[Bagby, Richardson, W.M . Henry, H.H . Holton,...",[],[],"[March 19th 1866, March 19 1866]","[19-March-1866, 19-March-1866]","[Bagby, Richardson, W.M . Henry, J, Edward Hom...",[],[],"[Bagby, Henry Signed, Richardsons, Edward Horn...",[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,[B.M . Richardson],[Raleigh],"[The Freedmens Bureau, Freedmens Court, Milita...",[Mon 28 . 1866],[28-Mon-1866],"[B.M, Richardson, Daniel Bagley Freedman, E. W...",[Raleigh],"[Freedmens Bureau, Bureau, Freedmens Court, Mi...",[],[],[Freedmens Bureau]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[The Freedmens Bureau],"[twenty one, twenty one year old]",[],"[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau],"[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau]
3,"one hundred dollar , two suit of clothes would...",[Genl Hawley],[],[],[],[],[Hawley],[],[],[Genl Hawley],[],[]
4,"question , I gave answer , I whipped.— All I s...","[Bagley, Richardson]",[Martain County],[],[],[],"[Bagley, Richardson]",[Martain County],[],[Bagley],[Martain County],[]


# list of ontonotes entity

In [47]:
listoto[:20]

[Span[0:3]: "one hundred dollar" → MONEY (1.0),
 Span[7:8]: "Bagby" → PERSON (1.0),
 Span[13:14]: "Richardson" → PERSON (1.0),
 Span[15:18]: "W.M . Henry" → PERSON (0.9998),
 Span[22:25]: "March 19th 1866" → DATE (1.0),
 Span[26:29]: "H.H . Holton" → PERSON (0.9969),
 Span[32:34]: "Edward Homer" → PERSON (1.0),
 Span[41:42]: "Richardsons" → PERSON (1.0),
 Span[45:46]: "Bagby" → PERSON (1.0),
 Span[69:71]: "Edward Horner" → PERSON (0.9999),
 Span[74:77]: "March 19 1866" → DATE (1.0),
 Span[77:80]: "H.H . Holton" → PERSON (0.9983),
 Span[82:84]: "Mathew Baldwin" → PERSON (1.0),
 Span[87:88]: "Bagly" → PERSON (1.0),
 Span[92:93]: "Richardson" → PERSON (1.0),
 Span[131:132]: "Mathew" → PERSON (0.9994),
 Span[135:138]: "H.H . Holton" → PERSON (0.9998),
 Span[0:3]: "The Freedmens Bureau" → ORG (0.9995),
 Span[4:7]: "B.M . Richardson" → PERSON (0.9995),
 Span[23:24]: "Raleigh" → GPE (0.9973)]

In [48]:
with open('ontoentity.pkl', 'wb') as f:
    pickle.dump(listoto, f)

# list of ner english entity

In [49]:
listeng[:20]

[Span[7:8]: "Bagby" → PER (0.9976),
 Span[17:19]: "Henry Signed" → PER (0.9605),
 Span[41:42]: "Richardsons" → PER (0.9903),
 Span[45:46]: "Bagby" → PER (0.9988),
 Span[69:71]: "Edward Horner" → PER (0.9942),
 Span[72:73]: "Sworn" → PER (0.9859),
 Span[82:84]: "Mathew Baldwin" → PER (0.9579),
 Span[87:88]: "Bagly" → PER (0.9998),
 Span[92:93]: "Richardson" → PER (0.9996),
 Span[137:138]: "Holton" → PER (0.9637),
 Span[1:3]: "Freedmens Bureau" → ORG (0.95),
 Span[1:3]: "Freedmens Bureau" → ORG (0.9782),
 Span[4:7]: "B. M. Richardson" → PER (0.994),
 Span[31:33]: "Daniel Bagley" → PER (0.9986),
 Span[44:46]: "Daniel Bagley" → PER (0.9997),
 Span[60:61]: "Richardson" → PER (0.9997),
 Span[73:75]: "Genl Hawley" → PER (0.9652),
 Span[79:81]: "Martain County" → LOC (0.9979),
 Span[87:88]: "Bagley" → PER (0.9901),
 Span[115:116]: "Bagley" → PER (0.9865)]

In [50]:
with open('nerenglargegentity.pkl', 'wb') as f:
    pickle.dump(listeng, f)

# list of ner english Large entity

In [51]:
listenglarge[:20]

[Span[7:8]: "Bagby" → PER (1.0),
 Span[13:14]: "Richardson" → PER (1.0),
 Span[15:18]: "W.M . Henry" → PER (0.915),
 Span[29:30]: "J" → PER (0.9986),
 Span[32:34]: "Edward Homer" → PER (1.0),
 Span[41:42]: "Richardsons" → PER (1.0),
 Span[45:46]: "Bagby" → PER (1.0),
 Span[69:71]: "Edward Horner" → PER (0.9961),
 Span[77:80]: "H.H . Holton" → PER (0.9247),
 Span[82:84]: "Mathew Baldwin" → PER (1.0),
 Span[87:88]: "Bagly" → PER (1.0),
 Span[92:93]: "Richardson" → PER (1.0),
 Span[131:132]: "Mathew" → PER (0.9999),
 Span[134:135]: "Baldwin" → PER (0.9999),
 Span[1:3]: "Freedmens Bureau" → ORG (1.0),
 Span[4:5]: "B.M" → PER (1.0),
 Span[6:7]: "Richardson" → PER (0.9999),
 Span[14:17]: "Daniel Bagley Freedman" → PER (0.9627),
 Span[18:19]: "Bureau" → ORG (0.9997),
 Span[23:24]: "Raleigh" → LOC (0.9999)]

In [52]:
with open('nerengentity.pkl', 'wb') as f:
    pickle.dump(listeng, f)